## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate  source  \
0  2025-07-26-02-25-05 +0000     nyt   
1  2025-07-26-02-25-01 +0000     bbc   
2  2025-07-26-02-24-05 +0000     bbc   
3  2025-07-26-02-23-22 +0000  nypost   
4  2025-07-26-02-00-00 +0000  nypost   

                                               title  \
0  New Reports on Russian Interference Show Trump...   
1  Starmer suggests UK will play role in new Gaza...   
2  Artist explores the toxic mining legacy of Zam...   
3  Democratic Texas congressional candidate dragg...   
4  Suffolk County Executive Ed Romaine signs off ...   

                                                link  
0  https://www.nytimes.com/2025/07/25/us/trump-ob...  
1     https://www.bbc.com/news/articles/cr5v5568mzro  
2     https://www.bbc.com/news/articles/cvgewwvp00yo  
3  https://nypost.com/2025/07/25/us-news/democrat...  
4  https://nypost.com/2025/07/25/us-news/suffolk-...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2445/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
5     trump     54
0       new     20
15     gaza     17
12     will     12
117  israel      9

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
175  2025-07-25-15-18-56 +0000     bbc   
158  2025-07-25-16-25-00 +0000     wsj   
43   2025-07-25-23-06-53 +0000    wapo   
206  2025-07-25-13-23-00 +0000     wsj   
130  2025-07-25-18-31-45 +0000  nypost   

                                                 title  \
175  Trump says Hamas 'didn't want deal' as US and ...   
158  President Trump has long wanted a U.S. soverei...   
43   Amid starvation scenes in Gaza, Trump administ...   
206  Trump Administration Sues New York City Over S...   
130  Elon Musk hits back at Trump after president s...   

                                                  link  score  
175     https://www.bbc.com/news/articles/crrqrz857wgo    114  
158  https://www.wsj.com/politics/policy/sovereign-...     98  
43   https://www.washingtonpost.com/national-securi...     93  
206  https://www.wsj.com/us-news/trump-administrati...     92  
130  https://nypost.com/2025/07/25/business/elon-mu...     87

## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate  source  \
175    114  2025-07-25-15-18-56 +0000     bbc   
94      72  2025-07-25-20-45-00 +0000     wsj   
28      45  2025-07-26-00-25-07 +0000     nyt   
158     44  2025-07-25-16-25-00 +0000     wsj   
85      44  2025-07-25-21-10-00 +0000     cbc   
53      38  2025-07-25-22-30-20 +0000  nypost   
21      36  2025-07-26-01-00-00 +0000     wsj   
286     32  2025-07-25-04-17-55 +0000  nypost   
3       30  2025-07-26-02-23-22 +0000  nypost   
39      29  2025-07-25-23-24-00 +0000     wsj   

                                                 title  \
175  Trump says Hamas 'didn't want deal' as US and ...   
94   Rep. Alexandria Ocasio-Cortez of New York stil...   
28   Thailand Warns of War With Cambodia as Deadly ...   
158  President Trump has long wanted a U.S. soverei...   
85   France plans to recognize Palestinian statehoo...   
53   Southwest flight plummets nearly 500 feet ‘in ...   
21   The Democratic Party’s image has eroded to its...   
286  George Santos bids dramatic farewell to suppor...   
3    Democratic Texas congressional candidate dragg...   
39   HHS Secretary RFK Jr. plans to remove all memb...   

                                                  link  
175     https://www.bbc.com/news/articles/crrqrz857wgo  
94   https://www.wsj.com/politics/policy/aoc-met-ga...  
28   https://www.nytimes.com/2025/07/25/world/asia/...  
158  https://www.wsj.com/politics/policy/sovereign-...  
85   https://www.cbc.ca/news/politics/france-macron...  
53   https://nypost.com/2025/07/25/us-news/southwes...  
21   https://www.wsj.com/politics/elections/democra...  
286  https://nypost.com/2025/07/25/us-news/george-s...  
3    https://nypost.com/2025/07/25/us-news/democrat...  
39   https://www.wsj.com/health/healthcare/rfk-heal...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
